GLM4的key的获取方式：https://e.naixuejiaoyu.com/p/t_pc/course_pc_detail/video/v_65dec9c6e4b064a8e8dee96b?product_id=p_65ba2d84e4b064a83b911697&content_app_id=&type=6

In [6]:
import os
import openai
from openai import OpenAI
import shutil

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re
import random
import string

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")

## pip install zhipuai

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

In [7]:
def sunwukong_function(data):
    """
    孙悟空算法函数，该函数定义了数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：sunwukong_function函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = df_new * 10
    return json.dumps(res.to_string())

In [8]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [119]:
def run_conversation(messages, functions_list=None, model="glm-4"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response.choices[0].message)
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [61]:
# 创建一个DataFrame
df = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]})

df_str = df.to_string()

In [120]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行孙悟空算法函数"}
]

In [63]:
functions_list=[sunwukong_function]

In [121]:
result = run_conversation(messages, functions_list=functions_list, model="glm-4")
print(result)

content='根据孙悟空算法函数的计算结果，数据集已经发生了变化。新的数据集如下：\n\n```\n   x1  x2\n0  10  30\n1  20  40\n```\n\n您可以根据这个结果进行后续的分析和处理。如果您需要进一步的帮助，请告诉我。' role='assistant' tool_calls=None
根据孙悟空算法函数的计算结果，数据集已经发生了变化。新的数据集如下：

```
   x1  x2
0  10  30
1  20  40
```

您可以根据这个结果进行后续的分析和处理。如果您需要进一步的帮助，请告诉我。


In [123]:
def chat_with_model(functions_list=None, 
                    prompt="你好", 
                    model="glm-4", 
                    system_message=[{"role": "system", "content": "你是小智助手。"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                                    functions_list=functions_list, 
                                    model=model)
        
        print(f"智能助手回答: {answer}")
        
        
        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

In [124]:
functions_list=[sunwukong_function]

In [125]:
chat_with_model(functions_list,prompt="你好")

智能助手回答: ，我是人工智能助手 ChatGLM，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。


您还有其他问题吗？(输入退出以结束对话):  你好，请介绍一下你自己？


智能助手回答: 我是小智助手，是一个人工智能语言模型，可以回答各种问题，提供各种帮助。


您还有其他问题吗？(输入退出以结束对话):  什么是机器学习？


智能助手回答: 机器学习（Machine Learning, ML）是人工智能（Artificial Intelligence, AI）的一个分支，它赋予了机器从数据中学习并做出决策或预测的能力，而无需每一步都进行明确的编程。机器学习算法通过各种数学模型来识别数据中的模式和规律，从而使机器能够对新数据做出预测或采取相应的行动。

机器学习可以分为几种主要类型：

1. **监督学习（Supervised Learning）**：在这种学习模式下，机器通过一个被称为训练集的数据集进行学习，其中包含了输入数据（特征）和对应的正确输出（标签）。目标是学习一个模型，能够对新的、未见过的输入数据做出准确的预测。

2. **无监督学习（Unsupervised Learning）**：在无监督学习中，机器处理的输入数据没有对应的标签，目标是发现数据中的结构或模式。聚类和关联规则学习是两种常见的无监督学习方法。

3. **半监督学习（Semi-supervised Learning）**：这种学习方法结合了监督学习和无监督学习的特点，其中一部分数据是标记的，但大部分数据是未标记的。

4. **强化学习（Reinforcement Learning）**：在强化学习中，机器（通常被称为智能体）通过与环境进行交互来学习如何完成特定任务。智能体接收到环境反馈，并根据这些反馈调整其行为以最大化某种累积奖励。

机器学习在许多领域都有应用，包括但不限于图像和语音识别、自然语言处理、医疗诊断、金融预测、推荐系统等。随着数据量的增加和计算能力的提升，机器学习已经成为解决复杂问题的重要工具。


您还有其他问题吗？(输入退出以结束对话):  退出
